In [1]:
import numpy as np
from scipy.optimize import linprog

In [5]:
def optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave):
    n, *_ = c.shape
    # slaiter's condition
    if not np.any(c_G.dot(x_wave) +\
                  0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G >= 0):
        raise Exception('Incopatible constraints')

    def F(x):
        return c.dot(x) + 0.5 * x.dot(D).dot(x)

    def f_grad(x):
        return c + D.dot(x)

    fx_wave_grad = f_grad(x_wave)
    active_idxs = np.argwhere(c_G.dot(x_wave) + 0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G == 0).ravel()

    A = c_G[active_idxs] + D_G[active_idxs].dot(x_wave)
    b = np.zeros(len(active_idxs))
    l_bounds = np.vstack((np.where(x_wave == 0, 0, None), np.ones(n))).T
    
    res = linprog(fx_wave_grad, A_ub=A, b_ub=b, bounds=l_bounds)
    l_star = res['x']

    # determine alpha
    a = f_grad(x_wave).dot(l_star)
    b = (x_hat - x_wave).dot(f_grad(x_wave))
    
    if b > 0:
        alpha = -a/(2*b)
    else:
        alpha = 1

    def x(t):
        return x_wave + t*(l_star + alpha * (x_hat - x_wave))

    def find_t():
        F_wave = F(x_wave)
        t = 1
        while not (F(x(t)) < F_wave and np.all(c_G.dot(x(t)) + 0.5 * x(t).dot(D_G).dot(x(t)) + alpha_G <= 0)):
            t /= 2
        return t

    return x(find_t())

## Пример 0

In [6]:
c = np.array([-3, -3])
D = np.array([
    [2, 1],
    [1, 2]
])

######### G ########
c_G = np.array([
    [1, -1],
    [-1, 1]
])
D_G = np.array([
    [[1, 0], [0, 1]],
    [[1, .5], [.5, 1]]
])
alpha_G = np.array([-1, -1.5])

x_hat = np.array([0, 0])
x_wave = np.array([0, 1])

optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave)

array([ 0.5   ,  0.5625])

## Пример 1

In [7]:
c = np.array([-1, -1, -1, -1, -2, 0, -2, -3])
B_0 = np.array([
    [2, 1, 0, 4, 0, 3, 0, 0],
    [0, 4, 0, 3, 1, 1, 3, 2],
    [1, 3, 0, 5, 0, 4, 0, 4]
])

D = B_0.T.dot(B_0)

######### G ########
c_G = np.array([
    [0, 60, 80, 0, 0, 0, 40, 0],
    [2, 0, 3, 0, 2, 0, 3, 0],
    [0, 0, 80, 0, 0, 0, 0, 0],
    [0, -2, 1, 2, 0, 0, -2, 1],
    [-4, -2, 6, 0, 4, -2, 60, 2]
])
B = np.array([
    [
        [0, 0, 0.5, 2.5, 1, 0, -2.5, -2],
        [0.5, 0.5, -0.5, 0, 0.5, -0.5, -0.5, -0.5],
        [0.5, 0.5, 0.5, 0, 0.5, 1, 2.5, 4]
    ],
    [
        [1, 2, -1.5, 3, -2.5, 0, -1, -0.5],
        [-1.5, -0.5, -1, -2.5, 3.5, -3, -1.5, -0.5],
        [1.5, 2.5, -1, 1, 2.5, 1.5, 3, 0, ]
    ],
    [
        [0.75, 0.5, -1, 0.25, 0.25, 0, 0.25, 0.75],
        [-1, 1, 4, 0.75, 0.75, 0.5, 7, -0.75],
        [0.5, -0.25, 0.5, 0.75, 0.5, 1.25, -0.75, -0.25]
    ],
    [
        [1.5, -1.5, -1.5, 2, 1.5, 0, 0.5, -1.5],
        [-0.5, -2.5, -0.5, -5, -2.5, 3.5, 1, 2],
        [-2.5, 1, -2, -1.5, -2.5, 0.5, 8.5, -2.5]
    ],
    [
        [1, 0.25, -0.5, 1.25, 1.25, -0.5, 0.25, -0.75],
        [-1., -0.75, -0.75, 0.5, -0.25, 1.25, 0.25, -0.5],
        [0, 0.75, 0.5, -0.5, -1., 1., -1., 1]
    ]
])
D_G = np.array([d.T.dot(d) for d in B])
alpha_G = np.array([-687.1250, -666.6250, -349.5938, -254.6250, -45.1563])

x_hat = np.array([0, 0, 0, 0, 0, 0, 0, 0])
x_wave = np.array([0, 8, 2, 1, 0, 4, 0, 0])

try:
    print(optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave))
except:
    print('already optim')

[ 0.  0.  0.  0.  0.  0.  0.  0.]


In [11]:
c = np.array([-1, -1, -1, -1, -2, 0, -2, -3])
B_0 = np.array([
    [2, 1, 0, 4, 0, 3, 0, 0],
    [0, 4, 0, 3, 1, 1, 3, 2],
    [1, 3, 0, 5, 0, 4, 0, 4]
])

D = B_0.T.dot(B_0)

c_G = np.array([
    [0, 60, 80, 0, 0, 0, 40, 0],
    [2, 0, 3, 0, 2, 0, 3, 0],
    [0, 0, 80, 0, 0, 0, 0, 0],
    [0, -2, 1, 2, 0, 0, -2, 1],
    [-4, -2, 6, 0, 4, -2, 60, 2]
])
B = np.array([
    [
        [0, 0, 0.5, 2.5, 1, 0, -2.5, -2],
        [0.5, 0.5, -0.5, 0, 0.5, -0.5, -0.5, -0.5],
        [0.5, 0.5, 0.5, 0, 0.5, 1, 2.5, 4]
    ],
    [
        [1, 2, -1.5, 3, -2.5, 0, -1, -0.5],
        [-1.5, -0.5, -1, 2.5, 3.5, 3, -1.5, -0.5],
        [1.5, 2.5, 1, 1, 2.5, 1.5, 3, 0]
    ],
    [
        [0.75, 0.5, -1, 0.25, 0.25, 0, 0.25, 0.75],
        [-1, 1, 1, 0.75, 0.75, 0.5, 1, -0.75],
        [0.5, -0.25, 0.5, 0.75, 0.5, 1.25, -0.75, -0.25]
    ],
    [
        [1.5, -1.5, -1.5, 2, 1.5, 0, 0.5, -1.5],
        [-0.5, -2.5, -0.5, -1, -2.5, 2.5, 1, 2],
        [-2.5, 1, -2, -1.5, -2.5, 0.5, 2.5, -2.5]
    ],
    [
        [1, 0.25, -0.5, 1.25, 1.25, -0.5, 0.25, -0.75],
        [-1, -0.75, -0.75, 0.5, -0.25, 1.25, 0.25, -0.5],
        [0, 0.75, 0.5, -0.5, -1, 1, -1, 1]
    ]
])

D_G = np.array([d.T.dot(d) for d in B])

alpha_G = np.array([-51.75, -436.75, -33, 7813. - 303.3750, -41.75])

x_hat = np.array([0, 0, 0, 0, 0, 0, 0, 0])
x_wave = np.array([1, 0, 0, 2, 4, 2, 0, 0])

c_G.dot(x_wave) + 0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G# print(optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave))

array([  0.00000000e+00,  -1.00000000e+01,   7.81250000e-01,
         7.71300000e+03,   0.00000000e+00])

In [22]:
c_G.dot(x_wave) +0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G

array([  0.00000000e+00,  -1.00000000e+01,   7.81250000e-01,
         7.71300000e+03,   0.00000000e+00])

In [23]:
c_G.dot(x_wave) +0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G

array([  0.00000000e+00,  -1.00000000e+01,   7.81250000e-01,
         7.71300000e+03,   0.00000000e+00])

In [24]:
c_G.dot(x_wave) +0.5 * x_wave.dot(D_G).dot(x_wave)

array([  51.75   ,  426.75   ,   33.78125,  203.375  ,   41.75   ])